In [1]:
import copy
from itertools import combinations

import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
from ipywidgets import interact
from numpy import arange, mean
from pylab import meshgrid, cm
%matplotlib inline

In [ ]:
def function(a,b):
    index = func_selector.options.index(func_selector.value)
    if index == 0:
        return (a-2)**4+(a-2*b)**2
    if index == 1:
        return -12*b+4*a**2+4*b**2-4*a*b
    if index == 2:
        return 8*a**2+4*a*b+5*b**2

In [3]:
def plot(population):
#     plt.clf()
    x = arange(-1,3,0.05)
    y = arange(-1,3,0.05)
    X,Y = meshgrid(x, y)
    Z = function(X, Y)
    fig, ax = plt.subplots(figsize=(10,10))
    im = ax.imshow(Z,cmap=cm.viridis,origin='lower',extent=(-1,3,-1,3))
    cset = ax.contour(Z,arange(0,500,1),linewidths=0.5,cmap=cm.Greys,extent=(-1,3,-1,3))
    ax.plot([i[0] for i in population],[i[1] for i in population],"o",color='red')
    plt.show()

In [9]:
class GeneticEvolution:
    def __init__(self, func, pop_size, mut_prob, cross_prob):
        self.func = func
        self.pop_size = pop_size
        self.population = []
        self.mutation_probability = mut_prob
        self.cross_prob = cross_prob
        self.history = []
        
    def build_plot(self, step):
        plot(self.history[step])

    # начальная случайная популяция
    def generate_random_population(self):
        self.population = ((np.random.rand(self.pop_size, 2)-0.5)*2).tolist()

    # инициализация поиска
    def initialize(self):
        self.history = []
        self.generate_random_population()

    # основная функция 
    def evolute(self, n_steps):
        n = 0
        progress.value = 0
        self.history.append(copy.copy(self.population))
        while n < n_steps:
            n+=1
            progress.value = 100*n_steps/n
            newpopulation = copy.copy(self.population)
            for comb in combinations(range(len(self.population)), 2):
                a = self.mutate(self.population[comb[0]])
                b = self.mutate(self.population[comb[1]])
                newitem = self.crossover(a, b)
                newpopulation.append(newitem)
            self.population = self.killing(newpopulation)
            self.history.append(copy.copy(self.population))
        return mean([i[0] for i in self.population]), mean([i[1] for i in self.population])

    def killing(self, population):
        res = np.argsort([self.func(*item) for item in population])
        res = res[:self.pop_size]
        return np.array(population)[res].tolist()

    def crossover(self, a, b):
        if np.random.rand() <= self.cross_prob:
            if np.random.rand() <= 0.5:
                return [a[0], b[1]]
            else:
                return [b[0], a[1]]
        else:
            if np.random.rand() <= 0.5:
                return [a[0], a[1]]
            else:
                return [b[0], b[1]]

    def mutate(self, a):
        if np.random.rand() <= self.mutation_probability:
            newa = a + (np.random.rand(2) - 0.5) * 0.05
        else:
            newa = a
        return newa



In [12]:
func_selector = widgets.Dropdown(
    options=['(a-2)^4+(a-2b)^2 [2;1]', '-12b+4a^2+4b^2-4ab [1;2]', '8a^2+4ab+5b^2 [0;0]'],
    description='Функция:'
)

population_size = widgets.IntText(
    value=50,
    description='Размер популяции:',
    disabled=False
)
steps_count = widgets.IntText(
    value=100,
    description='Кол-во шагов:',
    disabled=False
)
mutation_probability = widgets.FloatText(
    value=0.1,
    description='Вероятность мутации:',
    disabled=False
)
cross_probability = widgets.FloatText(
    value=0.5,
    description='Вероятность скрещивания:',
    disabled=False
)
start = widgets.Button(
    description='Старт'
)
progress = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    step=1,
    description='Прогресс:',
    bar_style='',
    orientation='horizontal'
)

def on_button_clicked(b):
    g = GeneticEvolution(function, 
                     population_size.value, 
                     mutation_probability.value, cross_probability.value)
    g.initialize()
    res = g.evolute(steps_count.value)
    interact(g.build_plot,step=widgets.IntSlider(min=0,max=len(g.history),step=1,value=0,description="Шаг:"))
    print(f'Результат оптимизации: {res[0]:.2f} {res[1]:.2f}')

display(func_selector)
start.on_click(on_button_clicked)
display(population_size)
display(steps_count)
display(mutation_probability)
display(cross_probability)
display(start)
display(progress)

Dropdown(description='Функция:', options=('(a-2)^4+(a-2b)^2 [2;1]', '-12b+4a^2+4b^2-4ab [1;2]', '8a^2+4ab+5b^2…

IntText(value=50, description='Размер популяции:')

IntText(value=100, description='Кол-во шагов:')

FloatText(value=0.1, description='Вероятность мутации:')

FloatText(value=0.5, description='Вероятность скрещивания:')

Button(description='Старт', style=ButtonStyle())

IntProgress(value=0, description='Прогресс:')

interactive(children=(IntSlider(value=0, description='Шаг:', max=101), Output()), _dom_classes=('widget-intera…

Результат оптимизации: -0.00 0.00
